# OSCordon Quick Start

In [ ]:
from lonboard import Map, PathLayer, PolygonLayer

from OSCordon.usrn_object import UsrnObject

## Example 1: Single Street with Bounding Box

This example shows how to:

1. Get data for one street using its USRN
2. Create a bound box around it 
3. Display both the street line and bounding box on a map

In [ ]:
# Simple Bounding Box and Single USRN
os_obj = UsrnObject()
usrn = "23009365"
street_data = os_obj.get_street_data(usrn)

bounds = os_obj.create_buffer(street_data, buffer_distance=10, return_geometry=False)

# Convert to WGS84 and return GeoDataFrame
# This has to be specified when working with just 1 USRN if you want the map to show
gdf = os_obj.convert_to_wgs84(street_data, bbox=bounds)

# Create layers for the USRN line and buffer geom
usrn_layer = PathLayer.from_geopandas(
    gdf[gdf["type"] == "usrn"], get_color=[0, 0, 255], width_min_pixels=3
)

boundary_layer = PolygonLayer.from_geopandas(
    gdf[gdf["type"] == "boundary"],
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Simple Bounding Box
Map(layers=[usrn_layer, boundary_layer])

## Example 2: Getting the Buffer Geometry

This shows how to get the actual buffer shape (polygon) instead of just the bounding box coordinates. 

This is useful when you need a precise buffer area for analysis.

Notice that `return_geometry` is set as True for this.

In [ ]:
buffer = os_obj.create_buffer(street_data, buffer_distance=10, return_geometry=True)
gdf = os_obj.convert_to_wgs84(street_data, bbox=buffer)

buffer_layer = PolygonLayer.from_geopandas(
    gdf[gdf["type"] == "boundary"],
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

Map(layers=[usrn_layer, buffer_layer])

## Example 3: Multiple Streets with Bounding Box

This example shows how to:

1. Work with multiple streets at once using a list of USRNs
2. Get a simple bounding box that covers all the streets
3. Display the streets and their overall boundary area

As a reminder, `return_geometry=False` gives you just the roads and a simple rectangle around them, which is good for getting a simple overview of an error or if you need to pass a bbox as a query param to another endpoint.

In [ ]:
# Single USRN
usrn_list = ["23012292", "23001950", "23002521"]

# Specify config
config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

gdf = os_obj.create_connected_route(
    usrn_list, buffer_distance=30, buffer_config=config, return_geometry=False
)

roads = gdf[gdf["type"] == "usrn"]
boundary = gdf[gdf["type"] == "route_bounds"]

# Create layers
road_layer = PathLayer.from_geopandas(
    roads,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

boundary_layer = PolygonLayer.from_geopandas(
    boundary,
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map
m = Map(layers=[road_layer, boundary_layer])
m

## Example 4: Multiple Streets with Detailed Buffers

This is the same as above, but with `return_geometry=True` to get:
1. Detailed buffer zones that follow the shape of each street
2. A merged buffer area that covers the entire route

This gives you more precise coverage areas.

In [ ]:
# Single USRN
usrn_list = ["23012292", "23001950", "23002521"]

# Specify config
config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

gdf = os_obj.create_connected_route(
    usrn_list,
    buffer_distance=30,
    buffer_config=config,
    merge_buffers=True,
    return_geometry=True,
)

roads = gdf[gdf["type"] == "usrn"]
boundary = gdf[gdf["type"] == "route_buffer"]

# Create layers
road_layer = PathLayer.from_geopandas(
    roads,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

boundary_layer = PolygonLayer.from_geopandas(
    boundary,
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map
m = Map(layers=[road_layer, boundary_layer])
m

## Example 5: Bounding Box for Dispersed Streets

This shows `create_proximity()` with `return_geometry=False` using streets that are close but not directly connected:
- Returns roads + single bounding box rectangle that encompasses all streets
- Shows how the bounding box covers the entire area even when streets are scattered
- Useful for getting the overall extent of multiple non-connected streets

In [ ]:
# Different USRNs - streets that are close but not directly connected
usrn_list_dispersed = ["23001950", "23032833", "23013026"]

# Get simple bounding box (return_geometry=False)
gdf_simple = os_obj.create_proximity(
    usrn_list_dispersed, buffer_distance=30, buffer_config=config, return_geometry=False
)

# Split into layers for simple view
roads_simple = gdf_simple[gdf_simple["type"] == "usrn"]
bounding_box = gdf_simple[gdf_simple["type"] == "route_bounds"]

print(
    f"Dispersed streets - Roads: {len(roads_simple)}, Bounding box: {len(bounding_box)}"
)

# Create map with bounding box
road_layer_simple = PathLayer.from_geopandas(
    roads_simple,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

bbox_layer = PolygonLayer.from_geopandas(
    bounding_box,
    get_fill_color=[255, 165, 0, 80],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map with roads and bounding box
m_simple = Map(layers=[road_layer_simple, bbox_layer])
m_simple

## Example 6: Individual Street Buffers

This shows `create_proximity()` with `return_geometry=True` using streets that are close but not directly connected:
- Returns roads + individual buffer polygons for each street
- Shows exact coverage area for each street

In [ ]:
usrn_list = ["23001950", "23032833", "23013026"]

config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

# Get individual buffers for each street (return_geometry=True)
gdf_detailed = os_obj.create_proximity(
    usrn_list, buffer_distance=30, buffer_config=config, return_geometry=True
)

# Split into layers for detailed view
roads_detailed = gdf_detailed[gdf_detailed["type"] == "usrn"]
individual_buffers = gdf_detailed[gdf_detailed["type"] == "buffer"]

print(
    f"Individual buffers - Roads: {len(roads_detailed)}, Buffers: {len(individual_buffers)}"
)

# Create map with individual buffers
road_layer = PathLayer.from_geopandas(
    roads_detailed,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

buffer_layer = PolygonLayer.from_geopandas(
    individual_buffers,
    get_fill_color=[255, 165, 0, 80],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map with roads and individual buffers
m = Map(layers=[road_layer, buffer_layer])
m

## Example 7: RoadLink-Based Routing

This demonstrates the RoadLinkObject class that provides more precise routing using individual road segments (roadlinks) instead of entire streets. 

This allows for:
- Creating routes that use only parts of streets
- Natural transitions at intersections

In [ ]:
from OSCordon.road_link_object import RoadLinkObject


async def demo_async_roadlinks():
    """Demonstrate async roadlink fetching with parallel API calls"""
    async with RoadLinkObject() as roadlink_obj:
        selected_roadlinks = [
            "e14bbfba-19e8-4423-b229-e3e751d5ceca",
            "1ead4c3d-473c-4d4b-9b6d-40e4ecbd7fa8",
            "9836d93e-8011-4056-8393-b69a6b090cc2",
            "465097be-885b-45a8-97ac-69441a78a5cf",
        ]

        buffer_config = {
            "cap_style": "round",
            "join_style": "mitre",
            "mitre_limit": 2.0,
            "resolution": 8,
            "single_sided": False,
        }

        print(f"Fetching {len(selected_roadlinks)} roadlinks in parallel...")

        gdf_roadlinks = await roadlink_obj.get_route_as_geodataframe_async(
            roadlink_ids=selected_roadlinks,
            buffer_distance=10,
            buffer_config=buffer_config,
            output_crs="EPSG:4326",
            batch_size=4,
        )

        return gdf_roadlinks


gdf_roadlinks = await demo_async_roadlinks()
roadlinks = gdf_roadlinks[gdf_roadlinks["type"] == "roadlink"]
buffer = gdf_roadlinks[gdf_roadlinks["type"] == "buffer"]

print(f"Route contains {len(roadlinks)} connecting roadlink segments")
print(f"USRNs detected: {roadlinks['usrn'].unique()}")
print(f"Road names: {roadlinks['road_name'].unique()}")

# Create map layers
roadlink_layer = PathLayer.from_geopandas(
    roadlinks,
    get_color=[0, 100, 255],
    width_min_pixels=4,
)

buffer_layer = PolygonLayer.from_geopandas(
    buffer,
    get_fill_color=[100, 200, 100, 80],
    get_line_color=[50, 150, 50],
    line_width_min_pixels=2,
)

Map(layers=[roadlink_layer, buffer_layer])